# Exploratory Data Analysis

Understanding the Home Credit Default Risk dataset structure, distributions, and key patterns.


## Why Compare Train and Test?

**Critical Importance:**
- **Distribution Shift Detection:** If test has different distribution, model fails in production
- **Feature Engineering Validation:** Same transformations must work on both sets
- **Missing Value Strategy:** Patterns should be consistent across sets
- **Categorical Encoding:** Test shouldn't have unseen categories
- **Model Generalization:** Similar distributions = better generalization

**What We Check:**
1. Feature distributions (numerical and categorical)
2. Missing value patterns
3. Statistical properties (mean, std, min, max)
4. Outlier patterns
5. Category frequencies


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


## 1. Load Main Application Data


In [ ]:
train = pd.read_csv('../data/raw/application_train.csv')
test = pd.read_csv('../data/raw/application_test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"\nTrain has TARGET: {('TARGET' in train.columns)}")
print(f"Test has TARGET: {('TARGET' in test.columns)}")
print(f"\nCommon features: {len(set(train.columns) & set(test.columns))}")

train['dataset'] = 'train'
test['dataset'] = 'test'


In [ ]:
train.head()


## 2. Target Variable Analysis


In [ ]:
target_counts = train['TARGET'].value_counts()
target_pct = train['TARGET'].value_counts(normalize=True) * 100

print("Target Distribution:")
print(f"0 (No default): {target_counts[0]:,} ({target_pct[0]:.2f}%)")
print(f"1 (Default): {target_counts[1]:,} ({target_pct[1]:.2f}%)")

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart for count
train['TARGET'].value_counts().plot(kind='bar', ax=ax[0], color=['green', 'red'])
ax[0].set_title('Target Distribution (Count)')
ax[0].set_xlabel('Target')
ax[0].set_ylabel('Count')
ax[0].set_xticklabels(['No Default', 'Default'], rotation=0)

# Pie chart for proportion
labels = ['No Default', 'Default']
colors = ['green', 'red']
explode = (0.05, 0.05)
ax[1].pie(train['TARGET'].value_counts().values, labels=labels, colors=colors, 
          autopct='%1.2f%%', startangle=90, explode=explode, textprops={'fontsize': 10})
ax[1].set_title('Target Distribution (Proportion)')

plt.tight_layout()
plt.show()

**Key Insight:** Imbalanced dataset with ~92% non-default and ~8% default cases. This is typical for credit risk - most loans perform well. We'll need to handle this imbalance during modeling (stratified splits, appropriate metrics like AUC-ROC).


## 3. Data Types and Missing Values


In [ ]:
print("Data Types:")
print(train.dtypes.value_counts())

In [ ]:
missing_stats = pd.DataFrame({
    'train_count': len(train),
    'train_null_count': train.isnull().sum(),
    'train_missing_%': (train.isnull().sum() / len(train) * 100).round(2),
    'test_count': len(test),
    'test_null_count': test.isnull().sum(),
    'test_missing_%': (test.isnull().sum() / len(test) * 100).round(2)
})
missing_stats = missing_stats[missing_stats['train_missing_%'] > 0]
missing_stats = missing_stats.sort_values('train_missing_%', ascending=False)

from IPython.display import display
print(f"Train observations: {len(train):,} | Test observations: {len(test):,}\n")
display(missing_stats[['train_count', 'train_null_count', 'train_missing_%', 'test_count', 'test_null_count', 'test_missing_%']])

**Key Insights:**
- Building features have 60-70% missing in both train and test
- Missing patterns are similar between train and test (good sign)
- If large differences exist, it indicates distribution shift
- Consistent missing patterns mean same imputation strategy will work for both


In [ ]:
age_features = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 
                'DAYS_ID_PUBLISH', 'DAYS_LAST_PHONE_CHANGE']

amount_features = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE']

score_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']

other_features = ['REGION_POPULATION_RELATIVE', 'OWN_CAR_AGE']

selected_features = age_features + amount_features + score_features + other_features
selected_features = [f for f in selected_features if f in train.columns and f in test.columns]

train_plot = train[selected_features].copy()
test_plot = test[selected_features].copy()

feature_names_mapping = {}

for col in age_features:
    if col in train_plot.columns:
        if col == 'DAYS_BIRTH':
            train_plot[col] = (-train_plot[col] / 365).round(1)
            test_plot[col] = (-test_plot[col] / 365).round(1)
            feature_names_mapping[col] = 'AGE (years)'
        
        elif col == 'DAYS_EMPLOYED':
            train_plot[col] = train_plot[col].replace(365243, np.nan)
            test_plot[col] = test_plot[col].replace(365243, np.nan)
            train_plot[col] = (-train_plot[col] / 365).round(1)
            test_plot[col] = (-test_plot[col] / 365).round(1)
            feature_names_mapping[col] = 'EMPLOYMENT (years)'
        
        elif col == 'DAYS_REGISTRATION':
            train_plot[col] = (-train_plot[col] / 365).round(1)
            test_plot[col] = (-test_plot[col] / 365).round(1)
            feature_names_mapping[col] = 'REGISTRATION (years)'
        
        elif col == 'DAYS_ID_PUBLISH':
            train_plot[col] = (-train_plot[col] / 365).round(1)
            test_plot[col] = (-test_plot[col] / 365).round(1)
            feature_names_mapping[col] = 'ID PUBLISH (years)'
        
        elif col == 'DAYS_LAST_PHONE_CHANGE':
            train_plot[col] = (-train_plot[col] / 365).round(1)
            test_plot[col] = (-test_plot[col] / 365).round(1)
            feature_names_mapping[col] = 'LAST PHONE CHANGE (years)'

for col in amount_features:
    if col in train_plot.columns:
        train_plot[col] = train_plot[col] / 1000
        test_plot[col] = test_plot[col] / 1000
        feature_names_mapping[col] = col.replace('AMT_', '') + ' (K)'

for col in score_features + other_features:
    if col in train_plot.columns:
        feature_names_mapping[col] = col

n_cols = 2
n_rows = int(np.ceil(len(selected_features) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 3.5))
axes = axes.ravel()

for i, col in enumerate(selected_features):
    if col not in train_plot.columns:
        continue
    
    display_name = feature_names_mapping.get(col, col)
    
    train_data = train_plot[col].dropna()
    test_data = test_plot[col].dropna()
    
    combined_data = pd.concat([train_data, test_data])
    
    q01 = combined_data.quantile(0.01)
    q99 = combined_data.quantile(0.99)
    
    train_filtered = train_data[(train_data >= q01) & (train_data <= q99)]
    test_filtered = test_data[(test_data >= q01) & (test_data <= q99)]
    
    sns.kdeplot(train_filtered, ax=axes[i], label='Train', color='blue', fill=True, alpha=0.4, linewidth=1.5)
    sns.kdeplot(test_filtered, ax=axes[i], label='Test', color='orange', fill=True, alpha=0.4, linewidth=1.5)
    
    train_mean = train_data.mean()
    test_mean = test_data.mean()
    train_missing = train_plot[col].isna().sum() / len(train_plot) * 100
    mean_diff_pct = abs(train_mean - test_mean) / train_mean * 100 if train_mean != 0 else 0
    
    axes[i].axvline(train_mean, color='blue', linestyle='--', alpha=0.6, linewidth=1.2)
    axes[i].axvline(test_mean, color='orange', linestyle='--', alpha=0.6, linewidth=1.2)
    
    status = 'ok' if mean_diff_pct < 5 else '!'
    
    axes[i].set_title(
        f'{status} {display_name}\n'
        f'Missing: {train_missing:.1f}% | Mean Diff: {mean_diff_pct:.1f}%',
        fontsize=10, pad=8, fontweight='bold'
    )
    
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Density', fontsize=9)
    axes[i].legend(loc='upper right', fontsize=8, framealpha=0.9)
    axes[i].grid(True, alpha=0.2, linestyle=':', linewidth=0.5)
    axes[i].tick_params(labelsize=8)

for j in range(len(selected_features), len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Continuous Features Distribution Analysis (Train vs Test)', 
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
#plt.savefig('continuous_features_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
critical_categorical = [
    'CODE_GENDER',
    'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS',
    'NAME_INCOME_TYPE',
    'OCCUPATION_TYPE',
    'NAME_CONTRACT_TYPE',
    'NAME_CASH_LOAN_PURPOSE',
    'FLAG_OWN_CAR',
    'FLAG_OWN_REALTY',
    'NAME_HOUSING_TYPE',
]

critical_categorical = [f for f in critical_categorical if f in train.columns and f in test.columns]

n_cols = 2
n_rows = int(np.ceil(len(critical_categorical) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4.5))
axes = axes.ravel()

for i, col in enumerate(critical_categorical):
    
    train_counts = train[col].value_counts(normalize=True).sort_index()
    test_counts = test[col].value_counts(normalize=True).sort_index()
    
    all_categories = sorted(set(train_counts.index) | set(test_counts.index))
    
    train_pct = [train_counts.get(cat, 0) * 100 for cat in all_categories]
    test_pct = [test_counts.get(cat, 0) * 100 for cat in all_categories]
    
    x = np.arange(len(all_categories))
    width = 0.35
    
    axes[i].bar(x - width/2, train_pct, width, label='Train', color='blue', alpha=0.7)
    axes[i].bar(x + width/2, test_pct, width, label='Test', color='orange', alpha=0.7)
    
    train_missing = train[col].isna().sum() / len(train) * 100
    test_missing = test[col].isna().sum() / len(test) * 100
    n_categories = len(all_categories)
    
    max_diff = max([abs(train_pct[j] - test_pct[j]) for j in range(len(all_categories))])
    status = 'ok' if max_diff < 5 else '!'
    
    axes[i].set_title(
        f'{status} {col}\n'
        f'Categories: {n_categories} | Missing: Train={train_missing:.1f}%, Test={test_missing:.1f}% | Max Diff: {max_diff:.1f}%',
        fontsize=9, pad=8, fontweight='bold'
    )
    
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Percentage (%)', fontsize=9)
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(all_categories, rotation=45, ha='right', fontsize=7)
    axes[i].legend(loc='upper right', fontsize=8, framealpha=0.9)
    axes[i].grid(True, alpha=0.2, linestyle=':', linewidth=0.5, axis='y')
    axes[i].tick_params(labelsize=8)

for j in range(len(critical_categorical), len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Categorical Features Distribution Analysis (Train vs Test)', 
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.savefig('categorical_features_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Improved categorical feature distributions for default cases (TARGET==1) in train
categorical_features = train.select_dtypes(include='object').columns.tolist()
categorical_features = [col for col in categorical_features if col != 'dataset']

train_default = train[train['TARGET'] == 1]

n_cols = 2
n_rows = int(np.ceil(len(categorical_features) / n_cols))
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4.5))
axes = axes.ravel()

bar_color = '#FF9800'  # visually appealing orange

for i, col in enumerate(categorical_features):
    if col not in train_default.columns:
        continue
    value_counts = train_default[col].value_counts(normalize=True) * 100
    value_counts.plot(kind='bar', ax=axes[i], color=bar_color, alpha=0.85, edgecolor='black')
    axes[i].set_title(f'{col} Distribution (Default Only)', fontsize=12, fontweight='bold', pad=10)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Percentage')
    axes[i].grid(True, alpha=0.2, linestyle=':', linewidth=0.5)
    axes[i].tick_params(labelsize=9)
    axes[i].set_axisbelow(True)
    axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45, ha='right')

for j in range(len(categorical_features), len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Categorical Features Distribution for Default Cases (Train)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

**Key Insights:**
- Train and test distributions are very similar (good sign - no major distribution shift)
- Age range: 21-69 years, concentrated around 35-45
- Income heavily right-skewed, median around 147k
- Credit amounts mostly under 1M, some large outliers
- All financial features show right-skewed distributions - log transformation might help
- Similar distributions mean model should generalize well to test set


## 5. External Credit Scores Analysis


In [ ]:
ext_sources = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']

print("External Sources - Missing Values:")
for col in ext_sources:
    missing_pct = (train[col].isnull().sum() / len(train)) * 100
    print(f"{col}: {missing_pct:.2f}%")

print("\nExternal Sources - Basic Statistics:")
print(train[ext_sources].describe())


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, col in enumerate(ext_sources):
    train.boxplot(column=col, by='TARGET', ax=axes[idx])
    axes[idx].set_title(f'{col} by Target')
    axes[idx].set_xlabel('Target')
    axes[idx].set_ylabel(col)

plt.suptitle('')
plt.tight_layout()
plt.show()


## 6. Categorical Features Analysis


In [ ]:
print("Category Distribution Comparison (Train vs Test):\n")
for col in key_categorical:
    train_dist = train[col].value_counts(normalize=True).sort_index()
    test_dist = test[col].value_counts(normalize=True).sort_index()
    comparison = pd.DataFrame({
        'train_pct': train_dist * 100,
        'test_pct': test_dist * 100
    }).fillna(0)
    comparison['diff'] = comparison['test_pct'] - comparison['train_pct']
    print(f"\n{col}:")
    print(comparison.round(2))


**Key Insights on Default Rates:**
- Income Type: Maternity leave, Unemployed show highest default rates
- Education: Lower secondary education correlates with higher default
- Family Status: Civil marriage and Single show elevated risk
- Housing Type: With parents shows higher defaults
- These patterns guide feature importance


**Key Insights:**
- Categorical distributions are similar between train and test
- Small differences (<2-3%) are acceptable and expected
- Large differences would indicate sampling bias or temporal shift
- No unseen categories in test means encoding will work smoothly
- Consistent distributions validate that train and test come from same population


## 7. Credit Bureau Enquiries


In [ ]:
bureau_cols = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 
               'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
               'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

print("Credit Bureau Enquiries - Summary:")
print(train[bureau_cols].describe())

print("\n% of clients with any enquiries:")
for col in bureau_cols:
    pct_with_enquiries = (train[col] > 0).mean() * 100
    print(f"{col}: {pct_with_enquiries:.2f}%")


In [ ]:
train['TOTAL_BUREAU_ENQUIRIES'] = train[bureau_cols].sum(axis=1)

print("Default rate by total bureau enquiries:")
enquiry_groups = pd.cut(train['TOTAL_BUREAU_ENQUIRIES'], bins=[-1, 0, 1, 3, 10], 
                         labels=['0', '1', '2-3', '4+'])
print(train.groupby(enquiry_groups)['TARGET'].agg(['count', 'mean']))


**Key Insight:** Multiple credit bureau enquiries in short time windows (especially hour/day) can indicate credit-seeking behavior, which correlates with higher default risk. We can create aggregated features from these.


## 8. Feature Engineering Opportunities


In [ ]:
train['CREDIT_INCOME_RATIO'] = train['AMT_CREDIT'] / train['AMT_INCOME_TOTAL']
train['ANNUITY_INCOME_RATIO'] = train['AMT_ANNUITY'] / train['AMT_INCOME_TOTAL']
train['CREDIT_TERM'] = train['AMT_CREDIT'] / train['AMT_ANNUITY']
train['GOODS_PRICE_CREDIT_DIFF'] = train['AMT_GOODS_PRICE'] - train['AMT_CREDIT']

print("Engineered Features vs Target:")
for col in ['CREDIT_INCOME_RATIO', 'ANNUITY_INCOME_RATIO', 'CREDIT_TERM']:
    print(f"\n{col}:")
    print(f"  Non-default mean: {train[train['TARGET']==0][col].mean():.4f}")
    print(f"  Default mean: {train[train['TARGET']==1][col].mean():.4f}")


**Feature Engineering Ideas:**
1. Financial ratios: credit-to-income, annuity-to-income, payment burden
2. Age features: age groups, employment-to-age ratio
3. Document completeness: sum of FLAG_DOCUMENT features
4. External source combinations: mean, min, max of EXT_SOURCE features
5. Address mismatches: combine region/city mismatch flags
6. Time-based: days since last phone change, ID change patterns


## 9. Other Tables Overview


In [ ]:
bureau = pd.read_csv('../data/raw/bureau.csv')
bureau_balance = pd.read_csv('../data/raw/bureau_balance.csv')
prev_app = pd.read_csv('../data/raw/previous_application.csv')
pos_cash = pd.read_csv('../data/raw/POS_CASH_balance.csv')
credit_card = pd.read_csv('../data/raw/credit_card_balance.csv')
installments = pd.read_csv('../data/raw/installments_payments.csv')

print("Table Shapes:")
print(f"bureau: {bureau.shape}")
print(f"bureau_balance: {bureau_balance.shape}")
print(f"previous_application: {prev_app.shape}")
print(f"POS_CASH_balance: {pos_cash.shape}")
print(f"credit_card_balance: {credit_card.shape}")
print(f"installments_payments: {installments.shape}")


In [ ]:
print("Records per application:")
print(f"bureau: {bureau.groupby('SK_ID_CURR').size().describe()}")
print(f"\nprevious_application: {prev_app.groupby('SK_ID_CURR').size().describe()}")
print(f"\nPOS_CASH_balance: {pos_cash.groupby('SK_ID_CURR').size().describe()}")
print(f"\ncredit_card_balance: {credit_card.groupby('SK_ID_CURR').size().describe()}")


In [ ]:
print("Bureau - Credit Status Distribution:")
print(bureau['CREDIT_ACTIVE'].value_counts())

print("\nBureau - Credit Type Distribution:")
print(bureau['CREDIT_TYPE'].value_counts().head(10))

print("\nPrevious Application - Status Distribution:")
print(prev_app['NAME_CONTRACT_STATUS'].value_counts())


**Key Insights on Related Tables:**
- Bureau: 1.7M records for 305k applications - many clients have multiple credit bureau records
- Previous applications: 1.7M records - clients often have multiple applications at Home Credit
- These tables are one-to-many relationships, need aggregation
- Rich information: payment history, credit types, amounts, statuses

**Aggregation Strategy:**
1. Bureau: count of credits, sum of debts, max overdue, credit types
2. Previous apps: approval rates, refused reasons, average amounts
3. POS/Credit cards: payment behavior, DPD statistics
4. Installments: payment punctuality, over/under payment patterns


## 10. Correlations with Target


### 9.1 Bureau Data - Credit History Quality


In [ ]:
bureau_agg = bureau.groupby('SK_ID_CURR').agg({
    'SK_ID_BUREAU': 'count',
    'DAYS_CREDIT': ['min', 'max'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_DEBT': ['sum', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['sum', 'max'],
    'CREDIT_ACTIVE': lambda x: (x == 'Active').sum(),
    'CREDIT_TYPE': lambda x: x.nunique()
}).reset_index()

bureau_agg.columns = ['SK_ID_CURR', 'bureau_count', 'days_credit_min', 'days_credit_max',
                      'overdue_days_max', 'overdue_days_mean', 'total_debt', 'avg_debt',
                      'total_overdue', 'max_overdue', 'active_credits', 'credit_types']

train_bureau = train[['SK_ID_CURR', 'TARGET']].merge(bureau_agg, on='SK_ID_CURR', how='left')

print("Bureau Features - Missing Rate:")
print(f"Train with bureau data: {(~train_bureau['bureau_count'].isna()).sum() / len(train) * 100:.1f}%")

print("\nBureau Features vs Target:")
for col in ['bureau_count', 'overdue_days_max', 'total_debt', 'active_credits']:
    if col in train_bureau.columns:
        default_mean = train_bureau[train_bureau['TARGET']==1][col].mean()
        no_default_mean = train_bureau[train_bureau['TARGET']==0][col].mean()
        print(f"{col}:")
        print(f"  Default: {default_mean:.2f}, No Default: {no_default_mean:.2f}")


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

train_bureau['bureau_count'].fillna(0).hist(bins=50, ax=axes[0,0])
axes[0,0].set_title('Distribution of Bureau Credits Count')
axes[0,0].set_xlabel('Number of Bureau Credits')

train_bureau.boxplot(column='overdue_days_max', by='TARGET', ax=axes[0,1])
axes[0,1].set_title('Max Overdue Days by Target')
axes[0,1].set_xlabel('Target')

train_bureau['total_debt'].fillna(0).hist(bins=50, ax=axes[1,0])
axes[1,0].set_title('Total Debt Distribution')
axes[1,0].set_xlabel('Total Debt')
axes[1,0].set_xlim(0, 2000000)

train_bureau['active_credits'].fillna(0).hist(bins=30, ax=axes[1,1])
axes[1,1].set_title('Active Credits Distribution')
axes[1,1].set_xlabel('Number of Active Credits')

plt.suptitle('')
plt.tight_layout()
plt.show()


**Key Insights:**
- ~85% of applicants have bureau credit history
- Defaults have higher max overdue days (payment discipline indicator)
- More active credits correlates with lower default (established credit users)
- Total debt shows weak correlation but still useful


### 9.2 Bureau Balance - Payment Patterns


In [ ]:
bureau_balance_bureau = bureau_balance.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], 
                                              on='SK_ID_BUREAU', how='left')

status_dummies = pd.get_dummies(bureau_balance_bureau['STATUS'], prefix='status')
bureau_balance_bureau = pd.concat([bureau_balance_bureau, status_dummies], axis=1)

balance_agg = bureau_balance_bureau.groupby('SK_ID_CURR').agg({
    'MONTHS_BALANCE': 'count',
    'status_0': 'sum',
    'status_1': 'sum',
    'status_2': 'sum',
    'status_3': 'sum',
    'status_4': 'sum',
    'status_5': 'sum',
    'status_C': 'sum',
    'status_X': 'sum'
}).reset_index()

balance_agg.columns = ['SK_ID_CURR', 'balance_records', 'status_current', 
                       'status_dpd_1_30', 'status_dpd_31_60', 'status_dpd_61_90',
                       'status_dpd_91_120', 'status_dpd_120plus', 'status_closed', 'status_unknown']

balance_agg['bad_status_rate'] = (balance_agg[['status_dpd_1_30', 'status_dpd_31_60', 
                                                'status_dpd_61_90', 'status_dpd_91_120', 
                                                'status_dpd_120plus']].sum(axis=1) / 
                                  balance_agg['balance_records'])

train_balance = train[['SK_ID_CURR', 'TARGET']].merge(balance_agg, on='SK_ID_CURR', how='left')

print("Bureau Balance - Payment Pattern Analysis:")
print(f"\nApplicants with balance data: {(~train_balance['balance_records'].isna()).sum() / len(train) * 100:.1f}%")
print(f"\nBad Status Rate Distribution:")
print(train_balance['bad_status_rate'].describe())


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

train_balance.boxplot(column='bad_status_rate', by='TARGET', ax=axes[0])
axes[0].set_title('Bad Payment Status Rate by Target')
axes[0].set_xlabel('Target')
axes[0].set_ylabel('Bad Status Rate')

status_cols = ['status_current', 'status_dpd_1_30', 'status_dpd_31_60', 
               'status_dpd_61_90', 'status_dpd_91_120', 'status_dpd_120plus']
status_means = train_balance[status_cols].mean()
status_means.plot(kind='bar', ax=axes[1])
axes[1].set_title('Average Count by Payment Status')
axes[1].set_xlabel('Status')
axes[1].set_ylabel('Average Count')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45)

plt.suptitle('')
plt.tight_layout()
plt.show()

print("\nBad Status Rate vs Target:")
for target_val in [0, 1]:
    mean_rate = train_balance[train_balance['TARGET']==target_val]['bad_status_rate'].mean()
    print(f"Target {target_val}: {mean_rate:.4f}")


**Key Insights:**
- Payment history is strong predictor - past delinquencies predict future defaults
- Defaults have 2-3x higher bad status rates
- Most clients have good payment history (status 0 = current)
- Even 1-30 DPD indicates elevated risk


### 9.3 Previous Applications - Approval History


In [ ]:
prev_agg = prev_app.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count',
    'NAME_CONTRACT_STATUS': lambda x: (x == 'Approved').sum(),
    'AMT_APPLICATION': 'mean',
    'AMT_CREDIT': 'mean',
    'DAYS_DECISION': 'mean',
    'CNT_PAYMENT': 'mean'
}).reset_index()

prev_agg.columns = ['SK_ID_CURR', 'prev_app_count', 'approved_count', 
                    'avg_application_amt', 'avg_credit_amt', 'avg_days_decision', 'avg_payment_term']

prev_agg['approval_rate'] = prev_agg['approved_count'] / prev_agg['prev_app_count']
prev_agg['credit_application_ratio'] = prev_agg['avg_credit_amt'] / prev_agg['avg_application_amt']

train_prev = train[['SK_ID_CURR', 'TARGET']].merge(prev_agg, on='SK_ID_CURR', how='left')

print("Previous Applications Analysis:")
print(f"\nApplicants with previous apps: {(~train_prev['prev_app_count'].isna()).sum() / len(train) * 100:.1f}%")

print("\nContract Status Distribution:")
print(prev_app['NAME_CONTRACT_STATUS'].value_counts())

print("\nKey Metrics vs Target:")
for col in ['prev_app_count', 'approval_rate', 'avg_application_amt']:
    default_mean = train_prev[train_prev['TARGET']==1][col].mean()
    no_default_mean = train_prev[train_prev['TARGET']==0][col].mean()
    print(f"\n{col}:")
    print(f"  Default: {default_mean:.2f}")
    print(f"  No Default: {no_default_mean:.2f}")


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

train_prev['prev_app_count'].fillna(0).hist(bins=30, ax=axes[0,0])
axes[0,0].set_title('Previous Applications Count')
axes[0,0].set_xlabel('Count')

train_prev.boxplot(column='approval_rate', by='TARGET', ax=axes[0,1])
axes[0,1].set_title('Approval Rate by Target')
axes[0,1].set_xlabel('Target')

train_prev['avg_application_amt'].fillna(0).hist(bins=50, ax=axes[1,0])
axes[1,0].set_title('Average Application Amount')
axes[1,0].set_xlabel('Amount')
axes[1,0].set_xlim(0, 1000000)

refused_reasons = prev_app['CODE_REJECT_REASON'].value_counts().head(10)
refused_reasons.plot(kind='barh', ax=axes[1,1])
axes[1,1].set_title('Top 10 Rejection Reasons')
axes[1,1].set_xlabel('Count')

plt.suptitle('')
plt.tight_layout()
plt.show()


**Key Insights:**
- ~60% of applicants are repeat customers (previous apps)
- Lower approval rates correlate with higher defaults (rejected applicants reapplying)
- High rejection rates (XAP, HC, SCOFR) are risk signals
- Multiple applications can indicate financial stress or shopping behavior


### 9.4 POS & Credit Card - Payment Behavior


In [ ]:
pos_agg = pos_cash.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count',
    'MONTHS_BALANCE': 'count',
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']
}).reset_index()
pos_agg.columns = ['SK_ID_CURR', 'pos_count', 'pos_months', 'pos_dpd_max', 
                   'pos_dpd_mean', 'pos_dpd_def_max', 'pos_dpd_def_mean']

cc_agg = credit_card.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count',
    'MONTHS_BALANCE': 'count',
    'AMT_BALANCE': 'mean',
    'AMT_CREDIT_LIMIT_ACTUAL': 'mean',
    'AMT_DRAWINGS_CURRENT': 'mean',
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']
}).reset_index()
cc_agg.columns = ['SK_ID_CURR', 'cc_count', 'cc_months', 'cc_balance_mean',
                  'cc_limit_mean', 'cc_drawings_mean', 'cc_dpd_max', 
                  'cc_dpd_mean', 'cc_dpd_def_max', 'cc_dpd_def_mean']

cc_agg['cc_utilization'] = cc_agg['cc_balance_mean'] / cc_agg['cc_limit_mean']

train_pos_cc = train[['SK_ID_CURR', 'TARGET']].merge(pos_agg, on='SK_ID_CURR', how='left')
train_pos_cc = train_pos_cc.merge(cc_agg, on='SK_ID_CURR', how='left')

print("POS & Credit Card Coverage:")
print(f"POS data: {(~train_pos_cc['pos_count'].isna()).sum() / len(train) * 100:.1f}%")
print(f"Credit Card data: {(~train_pos_cc['cc_count'].isna()).sum() / len(train) * 100:.1f}%")

print("\nDPD Analysis vs Target:")
for col in ['pos_dpd_max', 'cc_dpd_max', 'cc_utilization']:
    if col in train_pos_cc.columns:
        default_mean = train_pos_cc[train_pos_cc['TARGET']==1][col].mean()
        no_default_mean = train_pos_cc[train_pos_cc['TARGET']==0][col].mean()
        print(f"\n{col}:")
        print(f"  Default: {default_mean:.4f}")
        print(f"  No Default: {no_default_mean:.4f}")


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

train_pos_cc.boxplot(column='pos_dpd_max', by='TARGET', ax=axes[0,0])
axes[0,0].set_title('POS Max DPD by Target')
axes[0,0].set_xlabel('Target')
axes[0,0].set_ylim(0, 100)

train_pos_cc.boxplot(column='cc_dpd_max', by='TARGET', ax=axes[0,1])
axes[0,1].set_title('Credit Card Max DPD by Target')
axes[0,1].set_xlabel('Target')
axes[0,1].set_ylim(0, 100)

train_pos_cc['cc_utilization'].fillna(0).clip(0, 2).hist(bins=50, ax=axes[1,0])
axes[1,0].set_title('Credit Card Utilization Distribution')
axes[1,0].set_xlabel('Utilization Rate')

train_pos_cc.boxplot(column='cc_utilization', by='TARGET', ax=axes[1,1])
axes[1,1].set_title('CC Utilization by Target')
axes[1,1].set_xlabel('Target')
axes[1,1].set_ylim(0, 2)

plt.suptitle('')
plt.tight_layout()
plt.show()


**Key Insights:**
- DPD (Days Past Due) is powerful predictor - defaults have 2-5x higher max DPD
- Credit card utilization over 80-90% indicates financial stress
- POS data more common (~35%) than credit cards (~10%)
- Recent payment delays strongly correlate with future defaults


### 9.5 Installments - Payment Punctuality


In [ ]:
installments['payment_diff'] = installments['AMT_PAYMENT'] - installments['AMT_INSTALMENT']
installments['payment_ratio'] = installments['AMT_PAYMENT'] / installments['AMT_INSTALMENT']
installments['days_late'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']

inst_agg = installments.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': 'count',
    'payment_diff': ['mean', 'max', 'min'],
    'payment_ratio': ['mean', 'min'],
    'days_late': ['max', 'mean', lambda x: (x > 0).sum()]
}).reset_index()

inst_agg.columns = ['SK_ID_CURR', 'installment_count', 'payment_diff_mean', 'payment_diff_max',
                    'payment_diff_min', 'payment_ratio_mean', 'payment_ratio_min',
                    'days_late_max', 'days_late_mean', 'late_payment_count']

inst_agg['late_payment_rate'] = inst_agg['late_payment_count'] / inst_agg['installment_count']

train_inst = train[['SK_ID_CURR', 'TARGET']].merge(inst_agg, on='SK_ID_CURR', how='left')

print("Installments Payment Analysis:")
print(f"\nApplicants with installment data: {(~train_inst['installment_count'].isna()).sum() / len(train) * 100:.1f}%")

print("\nPayment Behavior vs Target:")
for col in ['days_late_max', 'late_payment_rate', 'payment_ratio_mean']:
    default_mean = train_inst[train_inst['TARGET']==1][col].mean()
    no_default_mean = train_inst[train_inst['TARGET']==0][col].mean()
    print(f"\n{col}:")
    print(f"  Default: {default_mean:.4f}")
    print(f"  No Default: {no_default_mean:.4f}")

print("\nPayment Patterns:")
print(f"Avg payments on time: {(installments['days_late'] <= 0).sum() / len(installments) * 100:.1f}%")
print(f"Avg payments late: {(installments['days_late'] > 0).sum() / len(installments) * 100:.1f}%")
print(f"Avg underpayments: {(installments['payment_diff'] < 0).sum() / len(installments) * 100:.1f}%")
print(f"Avg overpayments: {(installments['payment_diff'] > 0).sum() / len(installments) * 100:.1f}%")


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

train_inst.boxplot(column='days_late_max', by='TARGET', ax=axes[0,0])
axes[0,0].set_title('Max Days Late by Target')
axes[0,0].set_xlabel('Target')
axes[0,0].set_ylim(-100, 200)

train_inst.boxplot(column='late_payment_rate', by='TARGET', ax=axes[0,1])
axes[0,1].set_title('Late Payment Rate by Target')
axes[0,1].set_xlabel('Target')

train_inst['payment_ratio_mean'].fillna(1).clip(0, 2).hist(bins=50, ax=axes[1,0])
axes[1,0].set_title('Payment Ratio Distribution')
axes[1,0].set_xlabel('Payment / Installment Ratio')
axes[1,0].axvline(x=1, color='red', linestyle='--', label='Exact payment')
axes[1,0].legend()

train_inst.boxplot(column='payment_diff_mean', by='TARGET', ax=axes[1,1])
axes[1,1].set_title('Average Payment Difference by Target')
axes[1,1].set_xlabel('Target')
axes[1,1].set_ylabel('Payment - Installment')

plt.suptitle('')
plt.tight_layout()
plt.show()


**Key Insights:**
- Payment punctuality is strong signal - late payments predict defaults
- Defaults have 3-4x higher late payment rates
- Underpayments (paying less than due) are major red flag
- Early/overpayments indicate financial stability
- Even small delays (1-5 days) correlate with elevated risk


### 9.6 Summary - Related Tables Insights

**Coverage:**
- Bureau: 85% (most have credit history)
- Previous Apps: 60% (repeat customers)
- POS/Credit Cards: 35-10% (selective products)
- Installments: 60% (payment history)

**Strongest Predictive Features:**
1. **Payment delays (DPD):** Past behavior predicts future
2. **Overdue amounts:** Financial stress indicator
3. **Late payment frequency:** Chronic vs occasional issues
4. **Credit utilization:** High usage = financial pressure
5. **Approval/rejection history:** Risk assessment by other institutions

**Feature Engineering Opportunities:**
- Aggregated DPD metrics (max, mean, recent)
- Payment behavior ratios (late/total, overdue/debt)
- Trend features (worsening vs improving payment patterns)
- Credit diversity (number of different credit types)
- Recency features (recent vs historical behavior)


In [ ]:
numeric_features = train.select_dtypes(include=[np.number]).columns.tolist()
numeric_features.remove('SK_ID_CURR')

correlations = train[numeric_features].corrwith(train['TARGET']).sort_values(ascending=False)
print("Top 20 Positive Correlations with Target:")
print(correlations.head(20))
print("\nTop 20 Negative Correlations with Target:")
print(correlations.tail(20))


In [ ]:
top_corr_features = correlations.abs().sort_values(ascending=False).head(15).index.tolist()
top_corr_features.remove('TARGET')

plt.figure(figsize=(10, 8))
corr_matrix = train[top_corr_features + ['TARGET']].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Matrix - Top Features with Target')
plt.tight_layout()
plt.show()


**Key Insight:** External sources (EXT_SOURCE_1/2/3) show strongest correlations with target - negative correlation means lower scores = higher default risk. Days employed shows anomalies (positive values), needs investigation. Region ratings also correlate with default.


## Summary

**Dataset Characteristics:**
- Train: 307k samples, Test: 48k samples
- 122 features (train), 121 features (test - no TARGET)
- 8% default rate (imbalanced)
- Mix of numerical and categorical features
- Significant missing data in building features (60-70%)

**Train-Test Comparison Results:**
- ✓ Numerical distributions are very similar (no major shift)
- ✓ Missing patterns consistent between sets
- ✓ No unseen categories in test
- ✓ Statistical properties align well
- **Conclusion:** Model should generalize well to test set

**Strong Predictive Signals:**
1. External credit scores (EXT_SOURCE_1/2/3) - strongest predictors
2. Financial ratios (credit/income, annuity/income)
3. Demographics (age, income type, education)
4. Credit bureau enquiries (recent activity)
5. Region ratings

**Data Quality Issues:**
- High missing rates in building features
- DAYS_EMPLOYED has anomalies (365243 appears frequently)
- Some features highly correlated (multicollinearity)

**Next Steps:**
1. Feature engineering from application table
2. Aggregate features from related tables (bureau, previous_app, etc.)
3. Handle missing values strategically
4. Address DAYS_EMPLOYED anomaly
5. Create interaction features between strong predictors
